In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import tensorflow as tf
import pandas as pd
import pickle
import random
import json


In [2]:
intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_words = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [16]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype = object)
train_x = list(training[:,0])
train_y = list(training[:,1])

In [6]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [7]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [8]:
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)

Epoch 1/100
20/20 [==============================] - 0s 1ms/step - loss: 3.1433 - accuracy: 0.0208
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 3.0805 - accuracy: 0.0521
Epoch 3/100
20/20 [==============================] - 0s 2ms/step - loss: 3.0033 - accuracy: 0.1146
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 2.9405 - accuracy: 0.1458
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 2.8888 - accuracy: 0.0938
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 2.8294 - accuracy: 0.1562
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 2.7515 - accuracy: 0.2292
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 2.6550 - accuracy: 0.2604
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 2.5653 - accuracy: 0.2500
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 2.4214 - accuracy: 0.3021
Epoch 11/

20/20 [==============================] - 0s 2ms/step - loss: 0.2562 - accuracy: 0.9479
Epoch 84/100
20/20 [==============================] - 0s 2ms/step - loss: 0.2855 - accuracy: 0.9375
Epoch 85/100
20/20 [==============================] - 0s 3ms/step - loss: 0.1844 - accuracy: 0.9583
Epoch 86/100
20/20 [==============================] - 0s 2ms/step - loss: 0.2214 - accuracy: 0.9479
Epoch 87/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3789 - accuracy: 0.8958
Epoch 88/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3290 - accuracy: 0.9375
Epoch 89/100
20/20 [==============================] - 0s 2ms/step - loss: 0.2228 - accuracy: 0.9271
Epoch 90/100
20/20 [==============================] - 0s 2ms/step - loss: 0.2591 - accuracy: 0.9271
Epoch 91/100
20/20 [==============================] - 0s 2ms/step - loss: 0.2277 - accuracy: 0.9375
Epoch 92/100
20/20 [==============================] - 0s 4ms/step - loss: 0.2608 - accuracy: 0.9375
Epoch 93/100


In [10]:

model.save('chatbot.h5',model)
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "learning_chatbot.pkl", "wb" ) )